# Using Stock Market Data to Make Price Predictions
I'm fascinated by the stock market, and I'm learning how to work with neural networks. This is a project I've been working on in my own time, in the hopes I can eventually use it to make some money.

By using data from twelvedata.com's stock API, I implement the keras library's neural networks to make stock predicitons. 
## Methods
My thoughts on how to best prepare data for the neural network
### Inputs/Outputs Ideas
1. Input raw prices, output raw price predictions
2. Input price derivatives, output percentage growth at different times
3. Input average price over surrounding time periods, output raw price predictions

### Secondary Objectives
1. Input other stock indicators such as P/E, EMACS, etc.


## Importing Libraries

In [70]:
# Import necessary libraries
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
import sklearn as sk
import requests as rq
import datetime as dt
import json
from os import path, mkdir
from random import choice, sample


## Helper Functions

In [274]:
def getLocalApiKey():
    """Returns my personal api key from a private directory
    """
    f = open("../../../Private/twelveDataApiKey.txt")
    return f.readline()


def print_dict_tree(d, indent=0):
    """Print tree of keys in `dict` object.
    
    Prints the different levels of nested keys in a `dict` object. When there
    are no more dictionaries to key into, prints objects type and byte-size.

    Input
    -----
    d : dict
    """
    for key, value in d.items():
        print('    ' * indent + str(key), end=' ')
        if isinstance(value, dict):
            print(); print_dict_tree(value, indent+1)
        else:
            print(":", str(type(d[key])).split("'")[1], "-", str(len(str(d[key]))))

            
def getStock(apiKey: str, symbol: str, period: str, start_date=None, end_date=None, dp=3):
    '''Returns the json of stock symbol
    
    Requests stock values between start and end date. 
    If no start and end date are selected, it automatically chooses dates 
    from 1 year and 2 months ago.
    
    Input
    -----
    apiKey: your api key
    symbol: stock symbol
    period: the time interval for stock data. Check twelvedata.com for valid inputs
    start_date: yyyy-mm-dd
    end_date: yyyy-mm-dd
    '''
    if not start_date:
        start_date = dt.datetime.today() - dt.timedelta(days=365)
        start_date = str(start_date.strftime('%Y-%m-%d'))
    if not end_date:
        end_date = dt.datetime.today() - dt.timedelta(days=60)
        end_date = str(end_date.strftime('%Y-%m-%d'))
    
    # make API request
    ts = rq.get("https://api.twelvedata.com/time_series?symbol="+symbol+"&interval="+period+"&start_date="+start_date+"&end_date="+end_date+"&dp="+str(dp)+"&apikey="+apiKey).json()
    # Check to see if it is a valid time series. Incorrect values for the 
    # interval or start/end_dates will still return dictionaries, but they will be invalid.
    assert(ts["values"])
    return ts


def saveTS(ts):
    '''Saves a json dict of a stock time series as a .txt
    
    Will create a folder to put saved stock time series in. Future timeseries
    will be placed in this folder as well.
    
    Input
    -----
    ts: timeseries that you want saved
    '''
    symbolName = ts["meta"]["symbol"].replace("/", "")
    if not path.isdir('stockTimeSeries'):
        mkdir("stockTimeSeries")
    with open("stockTimeSeries/"+symbolName+'.txt', 'w') as outfile:
        json.dump(ts, outfile)
        
        
def formatData(ts: dict, backSearch=90, forecastPattern: list = [1, 5, 10, 20, 50]):
    '''Returns time series in a format that is usable by the keras neural network
    
    Returns a list of prices 
    '''
    
    # Ordered list of prices from most old to recent
    prices = [period["close"] for period in ts["values"]]
    prices.reverse()
    
    #print(prices)
    
    
    # Prevents trying to verify prices in unknown time periods
    nPrices = len(prices)
    gap = max(forecastPattern)
    maxIndex = nPrices - gap
    minIndex = backSearch
    indices = [i for i in range(minIndex, maxIndex)]
        
    X = []
    y = []
    
    # Allows us to add prices with their corresponding forecasts to new lists in a random order
    while indices:
        index = choice(indices)
        indices.remove(index)
        xBackSearch = []
        for i in range(backSearch):
            xBackSearch.append(float(prices[index-i]))
        X.append(xBackSearch)
        yForecast = []
        for delta in forecastPattern:
            yForecast.append(float(prices[index+delta]))
        y.append(yForecast)
    
    X = np.reshape(X, (-1, backSearch))
    y = np.reshape(y, (-1, len(forecastPattern)))
    # Prints price, followed by forecast
    
    return X, y
    

def splitData(X: list, y: list, ratio=.5):
    '''Returns 4 lists, composed to training and test data
    
    Input
    -----
    X: data for input neurons
    y: data for results
    
    Output
    ------
    '''
    split = int(len(X)*ratio)
    x_training = X[:split]
    y_training = y[:split]
    x_test = X[split:]
    y_test = y[split:]
    return x_training, y_training, x_test, y_test
    

def trainData(x_train, y_train, scalar=False, priceTiming="cls"):
    '''
    descriptioin
    
    Input
    -----
    priceTiming: Can be "avg", which averages closing price over surrounding time periods, or
    "cls", which uses closing price for that time period
    To implement: Averaging over course of day, averaging over course of weeks, etc
    
    Output
    ------
    X: A list of lists containing stock prices from the last 
    90 entries from an arbitrary point in time
    y: a list of lists containing the stock prices a number of 
    days in the future corresponding to theforecast pattern'''
    pass

## Using AAPL stock to train the neural network
It is uncertain whether all stocks behave in the same way. For example, a neural network learning from apple stock 
may not be able to be as accurate when predicting oil stocks, or even other tech sector stocks. However, in order to create the foundation of the neural network, I wanted to use it on a high volume, high market cap, relatively low volatility stock. If I can train it with AAPL stock, I may be able to apply the neural network to other stocks.

## Model 1
In model 1, I use raw data to make predictions. Even making predictions only 3 days in advance gives poor results.

In [283]:
# get and save the aapl stock time series as a .txt
myApiKey = getLocalApiKey()

aaplTS = getStock(myApiKey, "aapl", "1day", start_date="2002-01-01", end_date="2020-9-30")

X, y = formatData(aaplTS, backSearch=300, forecastPattern=[1, 2, 3])
x_train, y_train, x_test, y_test = splitData(X, y)
print(x_train[:10])
print(y_train[:10])


[[14.386 15.223 14.995 ... 17.024 16.744 16.57 ]
 [53.76  53.865 54.395 ... 35.568 36.072 36.29 ]
 [12.615 13.344 13.477 ...  9.141  9.397  9.427]
 ...
 [20.139 20.274 20.36  ... 22.594 23.022 23.207]
 [ 3.201  3.156  3.156 ...  5.855  5.943  5.868]
 [ 2.128  2.066  2.053 ...  1.325  1.272  1.263]]
[[14.002 13.948 14.238]
 [53.763 53.873 54.04 ]
 [13.358 12.989 13.346]
 [25.183 24.952 24.688]
 [ 3.26   3.208  3.109]
 [ 7.035  7.008  7.017]
 [ 4.875  4.709  4.83 ]
 [20.003 19.804 20.036]
 [ 3.24   3.364  3.321]
 [ 2.101  2.107  2.051]]


In [285]:
model = Sequential()
model.add(Dense(50, input_dim=len(x_train[0]), activation = 'relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(len(y_train[0]), activation='softmax'))

model.compile(optimizer='sgd',
              loss='mse',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=500, batch_size=len(x_train)//10)

score = model.evaluate(x_test, y_test, batch_size=len(x_train)//10)
print("accuracy:", score[1])

Epoch 1/500
11/11 [==============================] - 0s 1ms/step - loss: 856.4226 - accuracy: 0.3502
Epoch 2/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3849 - accuracy: 0.3933
Epoch 3/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3959 - accuracy: 0.3679
Epoch 4/500
11/11 [==============================] - 0s 2ms/step - loss: 856.3749 - accuracy: 0.3765
Epoch 5/500
11/11 [==============================] - 0s 2ms/step - loss: 856.3630 - accuracy: 0.3502
Epoch 6/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3566 - accuracy: 0.3566
Epoch 7/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3582 - accuracy: 0.3625
Epoch 8/500
11/11 [==============================] - 0s 2ms/step - loss: 856.3514 - accuracy: 0.3502
Epoch 9/500
11/11 [==============================] - 0s 2ms/step - loss: 856.3489 - accuracy: 0.3661
Epoch 10/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3527 - accura

11/11 [==============================] - 0s 1ms/step - loss: 856.3766 - accuracy: 0.3643
Epoch 82/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3513 - accuracy: 0.3611
Epoch 83/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3477 - accuracy: 0.3634
Epoch 84/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3491 - accuracy: 0.3339
Epoch 85/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3513 - accuracy: 0.3285
Epoch 86/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3461 - accuracy: 0.3770
Epoch 87/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3480 - accuracy: 0.3593
Epoch 88/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3488 - accuracy: 0.3394
Epoch 89/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3557 - accuracy: 0.3362
Epoch 90/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3781 - accuracy: 

11/11 [==============================] - 0s 1ms/step - loss: 856.3452 - accuracy: 0.3697
Epoch 161/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3449 - accuracy: 0.3743
Epoch 162/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3456 - accuracy: 0.3403
Epoch 163/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3455 - accuracy: 0.3530
Epoch 164/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3453 - accuracy: 0.3725
Epoch 165/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3452 - accuracy: 0.3253
Epoch 166/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3463 - accuracy: 0.2995
Epoch 167/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3454 - accuracy: 0.3326
Epoch 168/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3448 - accuracy: 0.3684
Epoch 169/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3448 - a

11/11 [==============================] - 0s 1ms/step - loss: 856.3458 - accuracy: 0.3697
Epoch 240/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3453 - accuracy: 0.3561
Epoch 241/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3453 - accuracy: 0.3570
Epoch 242/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3459 - accuracy: 0.3516
Epoch 243/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3455 - accuracy: 0.3543
Epoch 244/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3464 - accuracy: 0.2977
Epoch 245/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3463 - accuracy: 0.3348
Epoch 246/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3457 - accuracy: 0.3652
Epoch 247/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3458 - accuracy: 0.3466
Epoch 248/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3455 - a

11/11 [==============================] - 0s 1ms/step - loss: 856.3447 - accuracy: 0.3611
Epoch 319/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3447 - accuracy: 0.3657
Epoch 320/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3446 - accuracy: 0.3647
Epoch 321/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3447 - accuracy: 0.3661
Epoch 322/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3447 - accuracy: 0.3507
Epoch 323/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3462 - accuracy: 0.3806
Epoch 324/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3451 - accuracy: 0.3783
Epoch 325/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3499 - accuracy: 0.3348
Epoch 326/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3467 - accuracy: 0.3566
Epoch 327/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3474 - a

11/11 [==============================] - 0s 1ms/step - loss: 856.3467 - accuracy: 0.3634
Epoch 398/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3455 - accuracy: 0.3570
Epoch 399/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3456 - accuracy: 0.3575
Epoch 400/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3452 - accuracy: 0.3525
Epoch 401/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3453 - accuracy: 0.3462
Epoch 402/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3451 - accuracy: 0.3521
Epoch 403/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3452 - accuracy: 0.3376
Epoch 404/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3452 - accuracy: 0.3598
Epoch 405/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3448 - accuracy: 0.3611
Epoch 406/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3448 - a

11/11 [==============================] - 0s 1ms/step - loss: 856.3452 - accuracy: 0.3725
Epoch 477/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3443 - accuracy: 0.3765
Epoch 478/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3446 - accuracy: 0.3607
Epoch 479/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3444 - accuracy: 0.3602
Epoch 480/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3447 - accuracy: 0.3734
Epoch 481/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3444 - accuracy: 0.3647
Epoch 482/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3442 - accuracy: 0.3638
Epoch 483/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3441 - accuracy: 0.3666
Epoch 484/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3442 - accuracy: 0.3715
Epoch 485/500
11/11 [==============================] - 0s 1ms/step - loss: 856.3440 - a

### Model 1 Commentary
Despite a fairly deep neural network and training on data representing the last 100 days, the neural network struggles to find much pattern in the stock data. However, this may be explained by 